In [1]:
import numpy as np 
import pandas as pd
import cv2
import glob
import re

### Convert DLC detections to YOLO Detection format

Format :
.csv : 
Sr. No, Frame, Pos x, Pos y , Territory_id


MOT Format : 
FIle name , frame , track id, bb_left, bb_top, bb_width, bb_height, confidence, class id, class_name 

In [2]:
DATE = ['20230302', '20230303']
SESSION = ['SM_Lek1']#, 'SE_Lek1']
DRONE = ['P1D1', 'P1D2', 'P2D3', 'P2D4', 'P3D5', 'P3D6']

file_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/SpatialRegistration'
box_size = 80

In [3]:
for date in DATE:
    for session in SESSION:
        for drone in DRONE:
            internal_path = file_directory + '/' + date + '/' + session + '/' + drone
            files = glob.glob(internal_path + '/' + date + '_' + session + '_' + drone + '_*.csv')
            
            for file in files:
                df_dlc = pd.read_csv(file)

                # Create an empty dataframe with headers in YOLO detection format
                headers = ['file_path', 'frame', 'track_id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'confidence', 'class_id', 'class_name']
                df = pd.DataFrame(columns=headers).reset_index(drop=True)

                df['frame'] = df_dlc['frame'].astype(int)
                df[['bb_left', 'bb_top']] = df_dlc[['pos_x', 'pos_y']]
                df['file_path'] = internal_path
                df['confidence'] = 0.5
                df['class_id'] = 5
                df['class_name'] = 'territory'
                df['track_id'] = -1

                # DLC models were trained on a third of the video size
                df['bb_left'] *= 3
                df['bb_top'] *= 3

                # Convert keypoint to bounding box
                df['bb_left'] -= box_size/2
                df['bb_top'] -= box_size/2
                df['bb_width'] = box_size
                df['bb_height'] = box_size

                # Save modified file
                df.to_csv(file[:-4] + '_YOLO.csv', index=False)